In [ ]:
# Seems to output 79433 images
!mkdir -p /kaggle/temp/
!cd /kaggle/temp/ && unzip -qo /kaggle/input/painter-by-numbers/replacements_for_corrupted_files.zip
!cd /kaggle/temp/ && unzip -qn /kaggle/input/painter-by-numbers/train.zip

# start time: 10:48,end: between 10:56, 11:03
# start time: 13:37, end before: 13:46

In [ ]:
# For some reason some files are too large and it seems non-trivial to skip them...
# Remove extremely large entries
!find /kaggle/temp -type f -size +64M | sed 's/.*\///' | awk '{print "sed /,"$1"/d -i /kaggle/working/all_data_info.csv" }'> /kaggle/temp/replace.sh
# Move the .csv to somewhere writable
!cp /kaggle/input/painter-by-numbers/all_data_info.csv /kaggle/working/all_data_info.csv
!bash /kaggle/temp/replace.sh

# Overwrite images that cause decompression bomb error 
!find /kaggle/temp -type f -size +64M | awk '{print "cp -f "$1"s",$1 }' | sed 's/...jpgs/.jpg/' > /kaggle/temp/replace.sh
!bash /kaggle/temp/replace.sh
!find /kaggle/temp -type f -size +64M | awk '{print "cp -f "$1"s",$1 }' | sed 's/..jpgs/.jpg/' > /kaggle/temp/replace.sh
!bash /kaggle/temp/replace.sh

# Kaggle times out after 9 hours, so move the images to persistent storage
# and shrink them to fit, selecting 300 for min pixels because that's the size they get resized to initially
!mkdir -p /kaggle/working/train

!find /kaggle/temp -type f -size +1M | grep jpg$ | awk '{print "mogrify -resize 300x300^ -path /kaggle/working/train "$1 }' > /kaggle/temp/replace.sh
!bash /kaggle/temp/replace.sh
!ls /kaggle/working/train | awk '{print "rm -f /kaggle/temp/train/"$1}' > /kaggle/temp/remove-resized.sh
!bash /kaggle/temp/remove-resized.sh

In [ ]:
!find /kaggle/temp -type f -size +512k | grep jpg$ | awk '{print "mogrify -resize 300x300^ -path /kaggle/working/train "$1 }' > /kaggle/temp/replace.sh
!bash /kaggle/temp/replace.sh
!ls /kaggle/working/train | awk '{print "rm -f /kaggle/temp/train/"$1}' > /kaggle/temp/remove-resized.sh
!bash /kaggle/temp/remove-resized.sh

!find /kaggle/temp -type f -size +128k | grep jpg$ | awk '{print "mogrify -resize 300x300^ -path /kaggle/working/train "$1 }' > /kaggle/temp/replace.sh
!bash /kaggle/temp/replace.sh
!ls /kaggle/working/train | awk '{print "rm -f /kaggle/temp/train/"$1}' > /kaggle/temp/remove-resized.sh
!bash /kaggle/temp/remove-resized.sh

!find /kaggle/temp -type f | grep jpg$ | awk '{print "mv "$1" /kaggle/working/train"}' > /kaggle/temp/replace.sh
!bash /kaggle/temp/replace.sh

# !find /kaggle/temp -type f | grep jpg$ | awk '{print "mogrify -resize 300x300^ -path /kaggle/working/train "$1 }' > /kaggle/temp/replace.sh
# !bash -x /kaggle/temp/replace.sh
!find /kaggle/working/train -type f | grep jpg | awk '{print "identify -verbose -regard-warnings "$1">/dev/null"}' > /kaggle/temp/validate.sh
!bash /kaggle/temp/validate.sh

!zip -r output.zip /kaggle/working/*
# !ls /kaggle/working/train/ | wc -l

In [ ]:
from __future__ import print_function, division
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import os
import torch
import pandas as pd
# from skimage import io, transform
from skimage import transform
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

Dataset comes with a csv file with annotations which looks like this:

In [ ]:
artist_frame = pd.read_csv('/kaggle/input/painter-by-numbers/all_data_info.csv')
# Filter out just the training data
artist_train = artist_frame.loc[artist_frame['in_train']].reset_index(drop=True)
# Remove the train and test, vs train only field, and the is training field.
artist_train = artist_train.drop(['artist_group', 'in_train'], axis=1)
# Keep only one entry per file
artist_train = artist_train.drop_duplicates(subset=['new_filename']).reset_index(drop=True)
del artist_frame
artist_train.head()

In [ ]:
index_names = artist_train[artist_train ['pixelsx'] >= 18000].index
artist_train.drop(index_names, inplace = True)
artist_train = artist_train.reset_index(drop=True)

img_name = artist_train['new_filename'][0]
genre = artist_train['genre'][0]
style = artist_train['style'][0]

print('Image name: {}'.format(img_name))
print('Genre: {}'.format(genre))
print('Style: {}'.format(style))
artist_train.describe()

In [ ]:
# Pick a threshold that makes other no more common than the least common
genre_threshold = 1600

value_counts = artist_train['genre'].value_counts() # Specific column 
to_remove = value_counts[value_counts <= genre_threshold].index
artist_train['genre'].replace(to_remove, 'other', inplace=True)
artist_train = artist_train.reset_index(drop='true')
artist_train['genre'].value_counts().plot(kind='bar', title='genre')

In [ ]:
genre_dict = artist_train['genre'].drop_duplicates().reset_index(drop='true').to_dict()

def get_genre_key(val):
    for key, value in genre_dict.items():
         if val == value:
            return key
    return 1
print(genre_dict)

In [ ]:
# Pick a threshold that makes other no more common than the most common
style_threshold = 300

value_counts = artist_train['style'].value_counts() # Specific column 
to_remove = value_counts[value_counts <= style_threshold].index
artist_train['style'].replace(to_remove, 'other', inplace=True)
del to_remove
del value_counts
artist_train['style'].value_counts().plot(kind='bar', title='style')


In [ ]:
del artist_train

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

## Dataset class

torch.utils.data.Dataset is an abstract class representing a dataset. Your custom dataset should inherit Dataset and override the following methods:

    __len__ so that len(dataset) returns the size of the dataset.
    __getitem__ to support the indexing such that dataset[i] can be used to get iiith sample.

Let’s create a dataset class for our face landmarks dataset. We will read the csv in __init__ but leave the reading of images to __getitem__. This is memory efficient because all the images are not stored in the memory at once but read as required.

Sample of our dataset will be a dict {'image': image, 'artist': artist, 'genre': genre, 'style': style}. Our dataset will take an optional argument transform so that any required processing can be applied on the sample. We will see the usefulness of transform in the next section.

In [ ]:
!ls /kaggle/working 

In [ ]:
class WikiArtDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.artist_frame = pd.read_csv('/kaggle/input/painter-by-numbers/all_data_info.csv')
        self.artist_frame = self.artist_frame.loc[self.artist_frame['in_train']].reset_index(drop=True)
        # Remove the train and test, vs train only field, and the is training field.
        self.artist_frame = self.artist_frame.drop(['artist_group', 'in_train'], axis=1)
        
        # Pick a threshold that makes 10 categories of genre
        genre_threshold = 1600

        value_counts = self.artist_frame['genre'].value_counts() # Specific column 
        to_remove = value_counts[value_counts <= genre_threshold].index
        self.artist_frame['genre'].replace(to_remove, 'other', inplace=True)
        self.artist_frame = self.artist_frame.reset_index(drop=True)
        # Pick a threshold that makes 10 categories of style
        style_threshold = 300

        value_counts = self.artist_frame['style'].value_counts() # Specific column 
        to_remove = value_counts[value_counts <= style_threshold].index
        self.artist_frame['style'].replace(to_remove, 'other', inplace=True)
        self.artist_frame = self.artist_frame.reset_index(drop=True)
        index_names = self.artist_frame[self.artist_frame['pixelsx'] >= 18000].index
        self.artist_frame.drop(index_names, inplace = True)
        self.artist_frame = self.artist_frame.reset_index(drop=True)
        # Keep only one entry per file
        self.artist_frame = self.artist_frame.drop_duplicates(subset=['new_filename']).reset_index(drop=True)
        self.artist_frame = self.artist_frame.reset_index(drop=True)
        self.genre_dict = self.artist_frame['genre'].drop_duplicates().reset_index(drop='true').to_dict()
        self.root_dir = root_dir
        del to_remove
        del value_counts
        del style_threshold
        del genre_threshold
        self.transform = transform

    def __len__(self):
        self.artist_frame = self.artist_frame.reset_index(drop=True)
        return len(self.artist_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.root_dir,self.artist_frame['new_filename'][idx])
        # ensure image stays or changes to PIL image or tensor
        image = Image.open(img_name).convert('RGB')
        genre = self.artist_frame['genre'][idx]
        genre_key = int(get_genre_key(genre))
        style = self.artist_frame['style'][idx]
        sample = {'image': image, 'genre_key': genre_key}

        if self.transform:

            sample['image'] = self.transform(sample['image'])

        return sample

Let’s instantiate this class and iterate through the data samples. We will print the sizes of first 4 samples and show their artist, genre, and style

In [ ]:
wikiart_dataset = WikiArtDataset(csv_file='/kaggle/input/painter-by-numbers/all_data_info.csv',
                                    root_dir='/kaggle/working/train')

fig = plt.figure()
print(len(wikiart_dataset))
for i in range(len(wikiart_dataset)):
    sample = wikiart_dataset[i]

    print(i, sample['image'].size, sample['genre_key'])
    plt.imshow(sample['image'])
    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(sample['genre_key']))
    ax.axis('off')
    # show_landmarks(**sample)

    if i == 3:
        
        plt.show()
        break

## Compose transforms

Now, we apply the transforms on a sample.

Let’s say we want to rescale the shorter side of the image to 256 and then randomly crop a square of size 224 from it. i.e, we want to compose Rescale and RandomCrop transforms. torchvision.transforms.Compose is a simple callable class which allows us to do this.

In [ ]:
scale = transforms.Resize(256)
crop = transforms.RandomCrop(128)
composed = transforms.Compose([transforms.Resize(256),
                               transforms.RandomCrop(224)])


# Apply each of the above transforms on sample.
fig = plt.figure()
sample = wikiart_dataset[5]

for i, tsfrm in enumerate([scale, crop, composed]):
    transformed_sample = tsfrm(sample['image'])
    ax = plt.subplot(1, 3, i + 1)
    plt.tight_layout()
    ax.set_title(i)
    plt.imshow(transformed_sample)
    

plt.show()

## Iterating through the dataset

Let’s put this all together to create a dataset with composed transforms. To summarize, every time this dataset is sampled:

    An image is read from the file on the fly
    Transforms are applied on the read image
    Since one of the transforms is random, data is augmentated on sampling

We can iterate over the created dataset with a for i in range loop as before.

In [ ]:
transformed_dataset = WikiArtDataset(csv_file='/kaggle/input/painter-by-numbers/all_data_info.csv',
                                           root_dir='/kaggle/working/train',
                                           transform=transforms.Compose([
                                               # transforms.ToPILImage(),
                                               transforms.Resize(300),
                                               transforms.RandomCrop(300),
                                               transforms.RandomRotation(15),
                                               transforms.CenterCrop(250),
                                               transforms.RandomCrop(224),
                                               transforms.RandomHorizontalFlip(),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                               transforms.Lambda(lambda x : x + 0.1*torch.randn_like(x))
                                           ]))

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]
    
    print(i, sample['image'].size(), sample['genre_key'])

    if i == 3:
        break

## Train, Validation split
The transfer learning tutorial calls for a train dataset and a validation dataset, so we'll do that here

In [ ]:
import copy
import torch
import time
from torchvision import transforms, datasets, models, utils
# 70 % of data for training 
train_dataset_len = int(len(transformed_dataset)*0.7)
# Use the rest for validation
valid_dataset_len = len(transformed_dataset) - train_dataset_len
# Probably not the best approach transforming the validation dataset the same way as the training dataset
# but it's good enough for Cindy work
train_dataset, valid_dataset = torch.utils.data.random_split(transformed_dataset, (train_dataset_len, valid_dataset_len))
# pick batch size that's a power of 2 and is approx 1% of the respective dataset
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=384, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=256, shuffle=True, num_workers=4)
print(len(train_dataloader))
print(len(valid_dataloader))

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

These two major transfer learning scenarios look as follows:

- Finetuning the convnet: Instead of random initializaion, we initialize the network with a pretrained network, like the one that is trained on imagenet 1000 dataset. Rest of the training looks as usual.
- ConvNet as fixed feature extractor: Here, we will freeze the weights for all of the network except that of the final fully connected layer. This last fully connected layer is replaced with a new one with random weights and only this layer is trained.


## Visualize a few images

Let’s visualize a few training images so as to understand the data augmentations.

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    # Hard to see, so make it bigger
    plt.figure(figsize = (60,60))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
data1 = next(iter(train_dataloader))
image = data1['image']
genre = data1['genre_key']
print(data1['image'].shape)
# Make a grid from batch
out = utils.make_grid(image, nrow=16)

imshow(out, title=genre)

## Training the model

Now, let’s write a general function to train a model. Here, we will illustrate:

    Scheduling the learning rate
    Saving the best model

In the following, parameter scheduler is an LR scheduler object from torch.optim.lr_scheduler.

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader1 = train_dataloader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader1 = valid_dataloader

            running_loss = 0.0
            running_corrects = 0
            batch_idx = 0
            # Iterate over data.
            dataset_size = len(dataloader1)
            for data1 in iter(dataloader1):
                # print(data1)
                inputs = data1['image']
                labels = data1['genre_key']
                batch_idx = batch_idx+1
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                print(batch_idx, loss.item())
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes
            epoch_acc = running_corrects.double() / dataset_sizes

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Visualizing the model predictions

Generic function to display predictions for a few images

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

## Finetuning the convnet

Load a pretrained model and reset final fully connected layer.

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 13 1 for each genre.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = torch.nn.Linear(num_ftrs, len(genre_dict))

criterion = torch.nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)